In [ ]:
%matplotlib inline
from lvmtipo.actors import lvm
from lvmagp.images import Image
from plotimg import plot_images

await lvm.spec.agc.start()

filename = (await lvm.spec.agc.expose(0.0042)).flatten().unpack("*.filename") 
print(filename)
plot_images([Image.from_file(filename)])


In [ ]:
%matplotlib inline

from cluplus.proxy import invoke
from lvmtipo.actors import lvm
from lvmagp.images import Image
from plotimg import plot_images

await lvm.spec.agc.start()
await lvm.skye.agc.start()

exptime = 0.0042
rc = await invoke(lvm.skye.agc.expose(exptime), lvm.spec.agc.expose(exptime))
filenames = rc.flatten().unpack("*.filename")

print(filenames)
plot_images([Image.from_file(f) for f in filenames])